### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
#importar planilhas:
import pandas as pd
from pathlib import Path

#anexar arquivos ao e-mail e enviar:
import smtplib
import email.message
import mimetypes
from email.message import EmailMessage

#localizar pastas:
caminho = Path.cwd() #localizar pasta Projeto AutomacaoIndicadores
bases_de_dados = Path(caminho/Path(r'Bases de Dados')) #localizar dataframes

#dataframe com e-mails:
emails_df = pd.read_excel(fr'{bases_de_dados}\Emails.xlsx')
display(emails_df)

#dataframes de lojas e vendas:
lojas_df = pd.read_csv(fr'{bases_de_dados}\Lojas.csv', encoding='latin-1', sep = ';')
vendas_df = pd.read_excel(fr'{bases_de_dados}\Vendas.xlsx')

#juntar dataframe de vendas_df com lojas_df pros próximos dataframes a serem criados para cada loja:
vendas_lojas_df = lojas_df.merge(vendas_df, on = 'ID Loja')
display(vendas_lojas_df)

vendas_lojas_df.info() #conferir dataframe de vendas_lojas_df (tudo certo)

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,jovitrash+helena@gmail.com
1,Shopping Midway Mall,Alice,jovitrash+alice@gmail.com
2,Norte Shopping,Laura,jovitrash+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,jovitrash+manuela@gmail.com
4,Shopping União de Osasco,Valentina,jovitrash+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,jovitrash+sophia@gmail.com
6,Rio Mar Recife,Isabella,jovitrash+isabella@gmail.com
7,Salvador Shopping,Heloisa,jovitrash+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,jovitrash+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,jovitrash+julia@gmail.com


,ID Loja,Loja,Código Venda,Data,Produto,Quantidade,Valor Unitário,Valor Final
0,1,Iguatemi Esplanada,1,2019-01-01,Sapato Estampa,1,358,358
1,1,Iguatemi Esplanada,1,2019-01-01,Camiseta,2,180,360
2,1,Iguatemi Esplanada,1,2019-01-01,Sapato Xadrez,1,368,368
3,1,Iguatemi Esplanada,21,2019-01-02,Camisa Gola V Listrado,2,116,232
4,1,Iguatemi Esplanada,34,2019-01-02,Sapato Listrado,1,363,363
...,...,...,...,...,...,...,...,...
100994,25,Shopping Barra,69908,2019-12-25,Meia Liso,2,38,76
100995,25,Shopping Barra,69908,2019-12-25,Camisa,1,100,100
100996,25,Shopping Barra,69913,2019-12-25,Gorro,1,80,80
100997,25,Shopping Barra,69913,2019-12-25,Cinto Linho,1,248,248


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100999 entries, 0 to 100998
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   ID Loja         100999 non-null  int64         
 1   Loja            100999 non-null  object        
 2   Código Venda    100999 non-null  int64         
 3   Data            100999 non-null  datetime64[ns]
 4   Produto         100999 non-null  object        
 5   Quantidade      100999 non-null  int64         
 6   Valor Unitário  100999 non-null  int64         
 7   Valor Final     100999 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 6.9+ MB


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [2]:
def criar_tabela(loja, tabela_mae):
    
    '''Cria uma tabela restrita às vendas de apenas uma loja a partir de uma tabela com vendas de várias lojas.
    
    Parameters:
    loja(str): a loja em questão;
    tabela_mae(DataFrame): tabela contendo vendas de várias lojas.
    
    Returns:
    tabela_loja(DataFrame): tabela com vendas da loja. 
    '''
    
    #separar dataframe por lojas e criar uma lista com nomes de cada loja:
    lojas_agrupadas_df = tabela_mae.groupby('Loja')
    lista_lojas = tabela_mae['Loja'].unique()

    #pegar dataframe separado correspondente à loja:
    tabela_loja = lojas_agrupadas_df.get_group(loja)

    #tratar dataframe loja:
    tabela_loja = tabela_loja.drop(['ID Loja', 'Loja'], axis = 1)
    
    return tabela_loja

In [3]:
def calcular_indicadores(tabela_loja):
    
    '''Calcula os indicadores de uma loja a partir da tabela de vendas dela.
    
    Parameters:
    tabela_loja(DataFrame): a tabela em questão.
    
    Returns:
    lista_indicadores(list): uma lista contendo todos os indicadores do último dia e do último ano da loja. 
    '''
    
    tabela_loja = tabela_loja.sort_values('Data', ascending = False)

    #pegar último dia e último ano:
    ultimo_dia = tabela_loja.iloc[0]['Data']
    ultimo_ano = tabela_loja.iloc[0]['Data'].year

    #criar uma tabela para o último dia (filtrar)
    tabela_ultimo_dia = tabela_loja.loc[tabela_loja['Data'] == ultimo_dia]

    #calcular indicadores
    i1d = tabela_ultimo_dia['Valor Final'].sum() #faturamento do último dia
    i2d = len(tabela_ultimo_dia['Produto'].unique()) #diversidade de produtos do último dia
    valor_venda_dia = tabela_ultimo_dia.groupby('Código Venda').sum()
    i3d = int(valor_venda_dia['Valor Final'].mean()) #número de vendas do último dia

    #criar uma tabela para o ultimo ano (filtrar)
    tabela_ultimo_ano = tabela_loja.loc[tabela_loja['Data'].dt.year == ultimo_ano]

    #calcular indicadores
    i1a = tabela_ultimo_ano['Valor Final'].sum() #faturamento do último ano
    i2a = len(pd.unique(tabela_ultimo_ano['Produto'])) #diversidade de produtos do último ano
    valor_venda_ano = tabela_ultimo_ano.groupby('Código Venda').sum()
    i3a = int(valor_venda_ano['Valor Final'].mean()) #número de vendas do último dia
    
    #reordenar tabela para ordem crescente
    tabela_loja = tabela_loja.sort_values('Data') 

    #organizar indicadores em uma tupla:
    lista_indicadores = [i1d, i2d, i3d, i1a, i2a, i3a]

    return lista_indicadores

### Passo 3 - Salvar a planilha na pasta de backup

In [4]:
def salvar_planilha(tabela_loja, loja):    
    '''Salva tabela de uma loja como uma planilha de Excel.
    
    Parameters:
    tabela_loja(DataFrame): a tabela em questão;
    loja(str): nome da loja.
    
    Returns:
    str_loja_df(str): o caminho para a planilha da loja que foi salva no computador. 
    '''
    
    #pegar última data registrada e formatar corretamente:
    data_formatada = tabela_loja.iloc[-1]['Data'].strftime('%m_%d_')

    #criar pasta de backup Iguatemi Esplanada:
    pasta_loja_bkp = caminho/Path(r'Backup Arquivos Lojas')/'{}'.format(loja)

    #salvar planilha na pasta de backup:
    if pasta_loja_bkp.exists():
        tabela_loja.to_excel(str(pasta_loja_bkp) + '/{}{}.xlsx'.format(data_formatada, loja), index=False)
    else:
        pasta_loja_bkp.mkdir()
        tabela_loja.to_excel(str(pasta_loja_bkp) + '/{}{}.xlsx'.format(data_formatada, loja))

    str_loja_df = str(pasta_loja_bkp) + '/{}{}.xlsx'.format(data_formatada, loja)
    
    return str_loja_df

In [5]:
#exemplo de uso:
tabela_iguatemi = criar_tabela('Iguatemi Esplanada', vendas_lojas_df)

str_iguatemi_df = salvar_planilha(tabela_iguatemi, 'Iguatemi Esplanada')

tabela_iguatema = pd.read_excel(salvar_planilha(tabela_iguatemi, 'Iguatemi Esplanada'))
display(tabela_iguatema)

,Código Venda,Data,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,Sapato Estampa,1,358,358
1,1,2019-01-01,Camiseta,2,180,360
2,1,2019-01-01,Sapato Xadrez,1,368,368
3,21,2019-01-02,Camisa Gola V Listrado,2,116,232
4,34,2019-01-02,Sapato Listrado,1,363,363
...,...,...,...,...,...,...
3975,69899,2019-12-25,Sapato Xadrez,4,368,1472
3976,69903,2019-12-25,Terno Liso,5,720,3600
3977,69928,2019-12-25,Meia,1,30,30
3978,69944,2019-12-25,Gorro Xadrez,5,85,425


### Passo 4 - Calcular o indicador para 1 loja

In [6]:
def indicador_bateu_a_meta (indicador, meta):
    '''Avalia se um indicador bateu uma meta determinada.
    
    Parameters:
    indicador(int): valor do indicador em questão;
    meta(int): valor da meta a atingir.
    
    Returns:
    status(str): o status do indicador apresentado em formato HTML - um símbolo verde se o indicador bateu a meta
    e um símbolo vermelho se o indicador não bateu a meta.
    '''
    
    if indicador >= meta:
        status = '<p style="color:green">◙</p>' #retornar símbolo verde
    else:
        status = '<p style="color:red">◙</p>' #caso contrário retornar símbolo vermelho
    
    return status

#meta dos indicadores:

m1d = 1000 #meta de faturamento do último dia
m2d = 4 #meta de diversidade de produtos do último dia
m3d = 500 #meta de ticket médio do último dia

m1a = 165000 #meta de faturamento do último ano
m2a = 120 #meta de diversidade de produtos do último ano
m3a = 500 #meta de ticket médio do último ano

In [7]:
#continuação do exemplo do passo acima
i1d, i2d, i3d, i1a, i2a, i3a = calcular_indicadores(tabela_iguatemi)
print(i3d, i3a)

946 795


In [8]:
#escrever OnePages em formatação HTML:

one_page_dia = f'''
<table style="width:40%">
  <tr>
    <th></th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>{i1d}</td>
    <td>{m1d}</td>
    <td>{indicador_bateu_a_meta(i1d, m1d)}</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>{i2d}</td>
    <td>{m2d}</td>
    <td>{indicador_bateu_a_meta(i2d, m2d)}</td>
  </tr>
    <tr>
    <td>Indicador 3</td>
    <td>{i3d}</td>
    <td>{m3d}</td>
    <td>{indicador_bateu_a_meta(i3d, m3d)}</td>
  </tr>
</table>
'''

one_page_ano = f'''
<table style="width:40%">
  <tr>
    <th></th>
    <th>Valor Ano</th>
    <th>Meta Ano</th>
    <th>Cenário Ano</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>{i1a}</td>
    <td>{m1a}</td>
    <td>{indicador_bateu_a_meta(i1a, m1a)}</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>{i2a}</td>
    <td>{m2a}</td>
    <td>{indicador_bateu_a_meta(i2a, m2a)}</td>
  </tr>
    <tr>
    <td>Indicador 3</td>
    <td>{i3a}</td>
    <td>{m3a}</td>
    <td>{indicador_bateu_a_meta(i3a, m3a)}</td>
  </tr>
</table>
'''

### Passo 5 - Enviar por e-mail para o gerente

In [9]:
def enviar_email(gerente, data, loja, one_page_dia, one_page_ano, email_dest, arquivo):
    
    '''Envia e-mail contendo informações de uma loja para o gerente dela.
    
    Parameters:
    gerente(str): nome do gerente;
    data(str): última data registrada na planilha de vendas da loja;
    loja(str): nome da loja;
    one_page_dia(str): OnePage do último dia da loja, com indicadores;
    one_page_ano(str): OnePage do último ano da loja, com indicadores;
    email_dest(str): e-mail do destinatário (gerente);
    arquivo(str): nome do arquivo a ser anexado (planilha salva usando a function salvar_planilha).
    
    Returns:
    None (apenas faz print do status de envio, confirmando que o e-mail foi enviado).
    '''
    
    corpo_email = """
    <p><b>Bom dia, {}</b></p>
    <p></p>
    <p>O resultado de ontem (dia {}) da Loja {} foi:</p>
    {}
    {}
    <p>Segue em anexo a planilha com todos os dados para mais detalhes. Qualquer dúvida, estou à disposição.</p>
    <p></p>
    <p>Att.,</p>
    <p>Jovi Prata</p>
    """.format(gerente, data, loja, one_page_dia, one_page_ano)

    msg = EmailMessage()
    msg['Subject'] = "One Page Dia {} - {}".format(data, loja)
    msg['From'] = 'jovitrash@gmail.com' #e-mail de exemplo (usado apenas para testes)
    msg['To'] = '{}'.format(email_dest)
    password = 'emqbzkitmdfdtzxq' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')
    
    
    
    mime_type, encoding = mimetypes.guess_type(arquivo)
    with open(arquivo, 'rb') as fp:
        dados = fp.read()
    
    data_formatada = '{}_{}_'.format(data[:2], data[3:5])
    
    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1],
                       filename=r"{}{}.xlsx".format(data_formatada, loja))

    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    
    #credenciais de login para enviar o e-mail:
    s.login(msg['From'], password)
    s.send_message(msg)
    
    print('''E-mail com processo enviado:
    Nome do(a) gerente: {};
    E-mail do(a) gerente: {};
    Data da planilha: {};
    Loja: {}.'''.format(gerente, email_dest, data, loja))

### Passo 6 - Automatizar todas as lojas

In [10]:
#relembrando meta dos indicadores:

m1d = 1000 #meta_fat_ultimo_dia
m2d = 4 #meta_diversidade_produtos_ultimo_dia
m3d = 500 #meta_ticket_medio_ultimo_dia

m1a = 165000 #meta_fat_ultimo_ano
m2a = 120 #meta_diversidade_produtos_ultimo_ano
m3a = 500 #meta_ticket_medio_ultimo_ano

#separar dataframe contendo todas as lojas

lojas_agrupadas_df = vendas_lojas_df.groupby('Loja')
lista_lojas = vendas_lojas_df['Loja'].unique()

for loja in lista_lojas:
    tabela_loja = criar_tabela(loja, vendas_lojas_df) #criando a tabela da loja x
    
    arquivo_loja_df = salvar_planilha(tabela_loja, loja) #salvando como um arquivo excel
    
    i1d, i2d, i3d, i1a, i2a, i3a = calcular_indicadores(tabela_loja) #calculando e armazenando os indicadores em variáveis
    
    #escrever OnePages em formatação HTML:
    
    one_page_dia = f'''
<table style="width:40%">
  <tr>
    <th></th>
    <th>Valor Dia</th>
    <th>Meta Dia</th>
    <th>Cenário Dia</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>{i1d}</td>
    <td>{m1d}</td>
    <td>{indicador_bateu_a_meta(i1d, m1d)}</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>{i2d}</td>
    <td>{m2d}</td>
    <td>{indicador_bateu_a_meta(i2d, m2d)}</td>
  </tr>
    <tr>
    <td>Indicador 3</td>
    <td>{i3d}</td>
    <td>{m3d}</td>
    <td>{indicador_bateu_a_meta(i3d, m3d)}</td>
  </tr>
</table>
'''
    
    one_page_ano = f'''
<table style="width:40%">
  <tr>
    <th></th>
    <th>Valor Ano</th>
    <th>Meta Ano</th>
    <th>Cenário Ano</th>
  </tr>
  <tr>
    <td>Indicador 1</td>
    <td>{i1a}</td>
    <td>{m1a}</td>
    <td>{indicador_bateu_a_meta(i1a, m1a)}</td>
  </tr>
  <tr>
    <td>Indicador 2</td>
    <td>{i2a}</td>
    <td>{m2a}</td>
    <td>{indicador_bateu_a_meta(i2a, m2a)}</td>
  </tr>
    <tr>
    <td>Indicador 3</td>
    <td>{i3a}</td>
    <td>{m3a}</td>
    <td>{indicador_bateu_a_meta(i3a, m3a)}</td>
  </tr>
</table>
'''

    #definir variáveis restantes para enviar o e-mail:
    gerente = emails_df.loc[emails_df['Loja'] == loja]['Gerente'].item()
    email_dest = emails_df[emails_df['Loja'] == loja]['E-mail'].item()
    ultimo_dia = tabela_loja.iloc[-1]['Data'].strftime('%d/%m')
    
    enviar_email(gerente, ultimo_dia, loja, one_page_dia, one_page_ano, email_dest, arquivo_loja_df)

E-mail com processo enviado:
    Nome do(a) gerente: Helena;
    E-mail do(a) gerente: jovitrash+helena@gmail.com;
    Data da planilha: 25/12;
    Loja: Iguatemi Esplanada.
E-mail com processo enviado:
    Nome do(a) gerente: Alice;
    E-mail do(a) gerente: jovitrash+alice@gmail.com;
    Data da planilha: 26/12;
    Loja: Shopping Midway Mall.
E-mail com processo enviado:
    Nome do(a) gerente: Laura;
    E-mail do(a) gerente: jovitrash+laura@gmail.com;
    Data da planilha: 26/12;
    Loja: Norte Shopping.
E-mail com processo enviado:
    Nome do(a) gerente: Manuela;
    E-mail do(a) gerente: jovitrash+manuela@gmail.com;
    Data da planilha: 25/12;
    Loja: Shopping Iguatemi Fortaleza.
E-mail com processo enviado:
    Nome do(a) gerente: Valentina;
    E-mail do(a) gerente: jovitrash+valentina@gmail.com;
    Data da planilha: 26/12;
    Loja: Shopping União de Osasco.
E-mail com processo enviado:
    Nome do(a) gerente: Sophia;
    E-mail do(a) gerente: jovitrash+sophia@gmail.com

### Passo 7 - Criar ranking para diretoria

In [11]:
def tratar_p_reais(quantia):
    '''Faz o tratamento de um número para representar uma quantia em reais de forma correta.
    
    Parameters:
    quantia(int/float): número a ser formatado.
    
    Returns:
    quantia(str): o número convertido para string e formatado corretamente.'''
    
    quantia = '{:_.2f}'.format(quantia)
    quantia = quantia.replace('.', ',')
    quantia = quantia.replace('_', '.')
    return quantia

In [12]:
#separar vendas de todas as lojas em dataframes de cada loja contendo apenas a coluna de faturamento e data
lojas_agrupadas_df = vendas_lojas_df.groupby('Loja')
lista_lojas = vendas_lojas_df['Loja'].unique()

#armazenar dia numa variavel
tabela_loja_sample = criar_tabela(lista_lojas[0], vendas_lojas_df)

tabela_loja_sample = tabela_loja_sample.sort_values('Data', ascending = False)
ultimo_dia_geral = tabela_loja_sample.iloc[0]['Data']

#criar dicionário de ranking do último dia a ser convertido em dataframe:
lista_fats_dia = []
ranking_ultimo_dia = {'Loja': lista_lojas, 'Faturamento': lista_fats_dia}

#criar dicionário de ranking do último ano a ser convertido em dataframe:
lista_fats_ano = []
ranking_ultimo_ano = {'Loja': lista_lojas, 'Faturamento': lista_fats_ano}

#calcular faturamentos do último dia em comum de cada loja e do último ano em comum:
for loja in lista_lojas:
    tabela_loja = criar_tabela(loja, vendas_lojas_df) #criando a tabela da loja x
    tabela_loja = tabela_loja.sort_values('Data', ascending = False)
    
    ultimo_dia = tabela_loja.iloc[0]['Data']
    #se o dia for maior que o anteriormente armazenado:
    if ultimo_dia > ultimo_dia_geral:
        #drop vendas desse dia
        tabela_loja = tabela_loja[tabela_loja.Data != ultimo_dia]
        #e pegar i1d com função
        i1d = calcular_indicadores(tabela_loja)[0]
        
    #ou se for menor que o anteriormente armazenado:
    elif ultimo_dia < ultimo_dia_geral:
        #atualizar variavel ultimo_dia com a nova variavel
        ultimo_dia_geral = ultimo_dia
        #e pegar i1d com função
        i1d = calcular_indicadores(tabela_loja)[0]
    
    #ou no final de tudo se for realmente o último dia que todas as lojas venderam:
    else:
        i1d = calcular_indicadores(tabela_loja)[0]
    
    lista_fats_dia.append(i1d)
    
    
    #tratamento feito acima não necessário abaixo: improvável uma loja estar um ano atrasada quanto último ano de registro.
    ultimo_ano = tabela_loja.iloc[0]['Data'].year
    
    i1a = calcular_indicadores(tabela_loja)[3]
    lista_fats_ano.append(i1a)


#criar dataframe a partir do dicionário com lojas e faturamentos do último dia:
ranking_ultimo_dia_df = pd.DataFrame.from_dict(ranking_ultimo_dia)
#display(ranking_ultimo_dia_df)

#dizer melhor e pior loja do ultimo dia:
ranking_ultimo_dia_df_ordenado = ranking_ultimo_dia_df.sort_values('Faturamento')

pior_loja_ultimo_dia = ranking_ultimo_dia_df_ordenado.iloc[0, :].values.tolist()

melhor_loja_ultimo_dia = ranking_ultimo_dia_df_ordenado.iloc[-1, :].values.tolist()


#criar dataframe a partir do dicionário com lojas e faturamentos do último ano:
ranking_ultimo_ano_df = pd.DataFrame.from_dict(ranking_ultimo_ano)
#display(ranking_ultimo_ano_df)

ranking_ultimo_ano_df_ordenado = ranking_ultimo_ano_df.sort_values('Faturamento')

pior_loja_ultimo_ano = ranking_ultimo_ano_df_ordenado.iloc[0, :].values.tolist()

melhor_loja_ultimo_ano = ranking_ultimo_ano_df_ordenado.iloc[-1, :].values.tolist()


ultimo_dia_formatado = ultimo_dia_geral.strftime('%d/%m')
ultimo_ano_formatado = ultimo_dia_geral.year #linha desnecessaria pois ultimo_ano ja foi definido


#salvar planilhas dos rankings do último dia:

pasta_ranking_ultimo_dia = caminho/Path(r'Ranking das Lojas')

if pasta_ranking_ultimo_dia.exists():
    ranking_ultimo_dia_df.to_excel(str(pasta_ranking_ultimo_dia) + '/Ranking das Lojas Dia {}.xlsx'.format(ultimo_dia_geral.strftime('%d_%m'), index=False))
else:
    pasta_ranking_ultimo_dia.mkdir()
    ranking_ultimo_dia_df.to_excel(str(pasta_ranking_ultimo_dia) + '/Ranking das Lojas Dia {}.xlsx'.format(ultimo_dia_geral.strftime('%d_%m'), index=False))
                                   
str_ranking_ultimo_dia = str(pasta_ranking_ultimo_dia) + '/Ranking das Lojas Dia {}.xlsx'.format(ultimo_dia_geral.strftime('%d_%m'))

#salvar planilhas dos rankings do último ano:

pasta_ranking_ultimo_ano = caminho/Path(r'Ranking das Lojas')

if pasta_ranking_ultimo_ano.exists():
    ranking_ultimo_ano_df.to_excel(str(pasta_ranking_ultimo_ano) + '/Ranking das Lojas Ano {}.xlsx'.format(ultimo_ano_formatado, index=False))
else:
    pasta_ranking_ultimo_ano.mkdir()
    ranking_ultimo_ano_df.to_excel(str(pasta_ranking_ultimo_ano) + '/Ranking das Lojas Ano {}.xlsx'.format(ultimo_ano_formatado, index=False))

str_ranking_ultimo_ano = str(pasta_ranking_ultimo_dia) + '/Ranking das Lojas Ano {}.xlsx'.format(ultimo_ano_formatado)

### Passo 8 - Enviar e-mail para diretoria

In [13]:
def enviar_email_diretoria(diretoria, ultimo_dia, ultimo_ano, 
                           melhor_loja_ultimo_dia, pior_loja_ultimo_dia,
                           melhor_loja_ultimo_ano, pior_loja_ultimo_ano,
                           email_dest, arquivo_dia, arquivo_ano):
    
    '''Envia e-mail contendo informações de uma loja para o gerente dela.
    
    Parameters:
    diretoria(str): nome da diretoria;
    ultimo_dia(str): último dia em comum de vendas das lojas;
    ultimo_ano(str): último ano em comum de vendas das lojas;
    melhor_loja_ultimo_dia(str): nome da loja que mais arrecadou no último dia;
    pior_loja_ultimo_dia(str): nome da loja que menos arrecadou no último dia;
    melhor_loja_ultimo_ano(str): nome da loja que mais arrecadou no último ano;
    pior_loja_ultimo_ano(str): nome da loja que menos arrecadou no último ano;
    email_dest(str): e-mail do destinatário (diretoria);
    arquivo_dia(str): nome do arquivo do último dia a ser anexado (planilha salva usando a function salvar_planilha);
    arquivo_ano(str): nome do arquivo do último ano a ser anexado (planilha salva usando a function salvar_planilha).
    
    Returns:
    None (apenas faz print do status de envio, confirmando que o e-mail foi enviado).
    '''
    
    corpo_email = f"""
    <p><b>Bom dia, {diretoria}</b></p>
    <p>Segue em anexo as planilhas dos rankings do último dia ({ultimo_dia.strftime('%d/%m')}) e do último ano das lojas ({ultimo_ano}). Diante do que foi registrado
nos rankings, ressalta-se as seguintes informações sobre as lojas:</p>
    <p></p>
    <p>No último dia, {melhor_loja_ultimo_dia[0]} foi a que mais arrecadou, demonstrando um faturamento de R$ {tratar_p_reais(melhor_loja_ultimo_dia[1])} nesse dia.
Nesse mesmo dia, {pior_loja_ultimo_dia[0]} foi a que menos arrecadou, demonstrando um faturamento de R$ {tratar_p_reais(pior_loja_ultimo_dia[1])};</p>
    <p></p>
    <p>No último ano, {melhor_loja_ultimo_ano[0]} foi a que mais arrecadou, demonstrando um faturamento de R$ {tratar_p_reais(melhor_loja_ultimo_ano[1])} nesse ano.
Nesse mesmo ano, {pior_loja_ultimo_ano[0]} foi a que menos arrecadou, demonstrando um faturamento de R$ {tratar_p_reais(pior_loja_ultimo_ano[1])}.</p>
    <p></p>
    <p>Qualquer dúvida, estou à disposição.</p>
    <p></p>
    <p>Att.,</p>
    <p>Jovi Prata</p>
    """

    msg = EmailMessage()
    msg['Subject'] = "Rankings das Lojas"
    msg['From'] = 'jovitrash@gmail.com' #e-mail de exemplo (usado apenas para testes)
    msg['To'] = '{}'.format(email_dest)
    password = 'emqbzkitmdfdtzxq' 
    msg.add_header('Content-Type', 'text/html')
    msg.set_content(corpo_email, subtype='html')
    
    
    #anexar arquivo do último dia:
    
    mime_type, encoding = mimetypes.guess_type(arquivo_dia)
    with open(arquivo_dia, 'rb') as fp:
        dados = fp.read()
    
    ultimo_dia_formatado = ultimo_dia.strftime('%d_%m')

    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1],
                       filename=r"Ranking das Lojas Dia {}.xlsx".format(ultimo_dia_formatado))
    
    #anexar arquivo do último ano:
    
    mime_type, encoding = mimetypes.guess_type(arquivo_ano)
    with open(arquivo_ano, 'rb') as fp:
        dados = fp.read()

    msg.add_attachment(dados, maintype=mime_type.split("/")[0], subtype=mime_type.split("/")[1],
                       filename=r"Ranking das Lojas Ano {}.xlsx".format(ultimo_ano))

    
    s = smtplib.SMTP('smtp.gmail.com: 587')
    s.starttls()
    
    #credenciais de login para enviar o e-mail:
    s.login(msg['From'], password)
    s.send_message(msg)
    
    print('E-mail enviado.')

In [14]:
#definir variáveis restantes para enviar o e-mail:
diretoria = emails_df.iloc[-1, 1]
email_dest = emails_df.iloc[-1, 2]

enviar_email_diretoria(diretoria, ultimo_dia_geral, ultimo_ano_formatado,
                       melhor_loja_ultimo_dia, pior_loja_ultimo_dia, 
                       melhor_loja_ultimo_ano, pior_loja_ultimo_ano, 
                       email_dest, str_ranking_ultimo_dia, str_ranking_ultimo_ano)

E-mail enviado.
